# Overview and Notes

What are interesting scenarios for exploring correlations?
* Expression-dependency correlations for dependency genes that are druggable. For these, the expression correlates are informative for selection of sensitive cell lines/patients. In addition, causally related expression correlates are suggestive of possible resistance mechanisms or secondary targets for a drug combination.
* Dependency-dependency correlations for undruggable genes. For these, the correlates are suggestive of substitute targets, or combinations of targets. However, as combinations are considered the causal interpretation becomes more important.
* Expression-expression correlations. In this case one idea would be to explore the strength and sign of relationships from the data vs. assertions from curated databases/literature, particularly from perturbational experiments.
* Co-expression patterns. Another possibility is to discover patterns of relationships between genes, e.g. that expression of ubiquitin ligases is correlated with the expression of their targets.

Another interesting direction would be to use the information in the INDRA database to discover patterns of mechanistic relationships that predict correlations between dependencies of two genes, or between expression of a gene and dependency of another gene. The simplest possible application of this would be to look at pairwise relationships between two genes (could even extract from SIF), and test to see which types of relationships, and signs of relationships predicted, and amount of evidence predicted a) the sign of a correlation and b) the strength of the correlation.

If the matrix is made complete, i.e., if entries are included for having no known relationship, we may find that having no relationship is not predictive of a lack of a correlation.

Similarly, there will be many mechanistic relationships that yield detectable correlations.

One hypothesis that could be tested is whether downstream siblings were more likely to have large correlations to each other than with their ancestors, highlighting that correlations rarely pick up causal relations in this type of data.

Could do this with graph convolutional nets?

Another application would be to use the data to refine vague relationships extracted from text, or to address issues with assembly, a la distant supervision.

Another direction would be to cluster the correlation graph to find likely intermediates.

Or, to use the redundancy of certain gene families to explain why certain correlations do *not* show up, i.e., the gene knockout has no effect because it has a redundant copy.

# Preliminaries

In [18]:
import pickle
from os.path import join, expanduser
from collections import defaultdict, Counter
import numpy as np
import pandas as pd
from scipy import stats
from matplotlib import pyplot as plt
from indra.databases import hgnc_client
from indra.tools import assemble_corpus as ac

from depmap_analysis.scripts.depmap_script2 import main as run_depmap
from depmap_analysis.scripts.depmap_script2 import mito_file
from depmap_analysis.preprocessing import get_mitocarta_info

%matplotlib notebook

## Load transcript and dependency data and compute correlations


Key variables/functions:
* `ccle_df`. CCLE expression data, DataFrame.
* `crispr_df`. DepMap CRISPR gene effect (dependency) data, DataFrame.
* `crispr_corr`
* `crispr_z`
* `rnai_df`. DepMap combined RNAi data, DataFrame.
* `rna_corr`
* `rnai_z`
* `dep_z`
* `dep_df`. CRISPR and RNAi data combined as averaged z-scores, DataFrame.
* `dep_corr`. Correlation matrix between dep-dep, dep-expr, and expr-expr (though expr-expr correlations are only defined in this matrix for the genes also included in the dependency data).

### Working directory

In [19]:
#basedir = 'data/20q4'
basedir = '/Users/johnbachman/Dropbox/1johndata/Knowledge File/Biology/Research/Big Mechanism/datasets/depmap_analysis/depmap/21q1'
figdir = join(expanduser('~'), 'Dropbox/DARPA projects/papers/INDRA paper 2/figures/figure_panels')

### Cell line metadata

Load the cell line metadata which includes mappings between cell line identifiers used by different datasets.

In [20]:
cell_line_df = pd.read_csv(join(basedir, 'sample_info.csv'))
#cell_line_df = pd.read_csv('data/DepMap-2019q1-celllines_v2.csv')

In [21]:
cell_line_map = cell_line_df[['DepMap_ID', 'CCLE_Name']]
cell_line_map.set_index('CCLE_Name', inplace=True)
cell_line_map.head()

,DepMap_ID
CCLE_Name,
NIHOVCAR3_OVARY,ACH-000001
HL60_HAEMATOPOIETIC_AND_LYMPHOID_TISSUE,ACH-000002
CACO2_LARGE_INTESTINE,ACH-000003
HEL_HAEMATOPOIETIC_AND_LYMPHOID_TISSUE,ACH-000004
HEL9217_HAEMATOPOIETIC_AND_LYMPHOID_TISSUE,ACH-000005


### RNAi data

Load combined RNAi data, normalize column names, map to DepMap cell line IDs, and drop duplicate columns.

In [22]:
%time rnai_df = pd.read_csv(join(basedir, 'D2_combined_gene_dep_scores.csv'), index_col=0)
rnai_df = rnai_df.transpose()
gene_cols = ['%s' % col.split(' ')[0] for col in rnai_df.columns]
rnai_df.columns = gene_cols
rnai_df = rnai_df.join(cell_line_map)
rnai_df = rnai_df.set_index('DepMap_ID')
# Drop duplicate columns
rnai_df = rnai_df.loc[:,~rnai_df.columns.duplicated()]

CPU times: user 2.21 s, sys: 190 ms, total: 2.4 s
Wall time: 2.42 s


In [23]:
rnai_df.head()

,A1BG,NAT2,ADA,CDH2,AKT3,MED6,NR2E3,NAALAD2,DUXB,PDZK1P1,...,RCE1,HNRNPDL,DMTF1,PPP4R1,CDH1,SLC12A6,KCNE2,DGCR2,CASP8AP2,SCO2
DepMap_ID,,,,,,,,,,,,,,,,,,,,,
ACH-001270,NaN,NaN,NaN,-0.194962,-0.256108,-0.174220,-0.140052,NaN,NaN,NaN,...,-0.201644,-0.363670,0.184260,-0.115616,-0.125958,NaN,0.088853,NaN,-0.843295,NaN
ACH-001000,NaN,NaN,NaN,-0.028171,0.100751,-0.456124,-0.174618,NaN,NaN,NaN,...,0.074889,0.152158,0.036011,0.117300,0.101725,NaN,-0.110628,NaN,-0.307031,NaN
ACH-001001,0.146042,0.102854,0.168839,0.063047,-0.008077,-0.214376,-0.153619,0.133830,0.138673,0.030345,...,0.006735,-0.033385,0.197651,-0.016372,0.077486,0.106165,0.057286,0.025596,-0.413669,0.122669
ACH-002319,-0.190388,0.384106,-0.120700,-0.237251,0.060267,-0.338946,-0.057551,0.134511,NaN,0.144463,...,0.209009,-0.156839,-0.155837,-0.001141,NaN,0.227968,0.028095,-0.080611,-1.849696,-0.078856
ACH-001827,0.907063,0.403192,0.004394,-0.017059,-0.094749,-0.328074,-0.089573,0.362029,NaN,-0.098161,...,-0.137465,-1.037848,-0.261262,-0.228016,NaN,0.088744,0.159467,0.014071,-0.414154,0.032661


In [24]:
recalculate = False
filename = 'rnai_correlations'
filepath = join(basedir, filename)
if recalculate:
    %time rnai_corr = rnai_df.corr()
    rnai_corr.to_hdf('%s.h5' % filepath, filename)
else:
    rnai_corr = pd.read_hdf('%s.h5' % filepath)

INFO: [2021-03-04 11:24:03] numexpr.utils - Note: NumExpr detected 12 cores but "NUMEXPR_MAX_THREADS" not set, so enforcing safe limit of 8.
INFO: [2021-03-04 11:24:03] numexpr.utils - NumExpr defaulting to 8 threads.


In [25]:
rnai_mean = rnai_corr.values.mean()
rnai_sd = rnai_corr.values.std()
rnai_z = (rnai_corr - rnai_mean) / rnai_sd

In [26]:
rnai_z.head()

,A1BG,NAT2,ADA,CDH2,AKT3,MED6,NR2E3,NAALAD2,DUXB,PDZK1P1,...,RCE1,HNRNPDL,DMTF1,PPP4R1,CDH1,SLC12A6,KCNE2,DGCR2,CASP8AP2,SCO2
A1BG,12.735562,-0.770498,-0.828444,0.593730,-0.334519,-0.076705,1.054949,0.591804,0.385034,-0.120219,...,1.446480,-1.964996,-0.403560,0.402625,-0.396057,0.131028,0.227373,-0.701314,0.218191,1.198913
NAT2,-0.770498,12.735562,0.572062,-0.052426,-0.528069,0.307443,-0.021474,-0.590996,-0.318764,-1.449979,...,0.371740,-0.754718,0.017751,0.024919,-2.128375,-0.190518,0.608389,-0.842078,-1.303389,-0.503340
ADA,-0.828444,0.572062,12.735562,-0.451619,-0.577117,0.422290,-0.188984,-0.570248,-0.078767,0.119471,...,0.475397,0.045575,0.639884,0.858577,-0.315503,-0.353140,-0.704099,1.271989,0.150168,-0.217580
CDH2,0.593730,-0.052426,-0.451619,12.735562,0.565031,-0.946426,-0.135411,0.118138,0.201983,-0.383189,...,-0.570892,-1.780034,1.163082,-1.070628,-0.127555,-0.228329,0.010906,-1.423871,-1.132664,-0.382607
AKT3,-0.334519,-0.528069,-0.577117,0.565031,12.735562,-0.378659,-1.136175,0.590426,0.837794,-0.574912,...,-1.262663,0.259384,0.443690,-0.919940,0.283450,-1.192266,-0.581840,-1.243842,-0.184748,-0.345918


### CRISPR Data

Load CRISPR dependency data and normalize column names to gene names.

In [27]:
%time crispr_df = pd.read_csv(join(basedir, 'Achilles_gene_effect.csv'), index_col=0)
gene_cols = ['%s' % col.split(' ')[0] for col in crispr_df.columns]
crispr_df.columns = gene_cols
# Drop any duplicate columns (shouldn't be any for CRISPR, but just in case)
crispr_df = crispr_df.loc[:,~crispr_df.columns.duplicated()]

CPU times: user 13.2 s, sys: 348 ms, total: 13.6 s
Wall time: 13.7 s


In [28]:
crispr_df.head()

,A1BG,A1CF,A2M,A2ML1,A3GALT2,A4GALT,A4GNT,AAAS,AACS,AADAC,...,ZWILCH,ZWINT,ZXDA,ZXDB,ZXDC,ZYG11A,ZYG11B,ZYX,ZZEF1,ZZZ3
DepMap_ID,,,,,,,,,,,,,,,,,,,,,
ACH-000004,0.164686,0.094628,-0.190746,-0.009197,0.032529,-0.191355,0.355279,-0.442430,0.298153,0.170712,...,-0.125219,-0.463670,0.031849,-0.007470,0.261947,0.237661,-0.420546,0.277953,0.227172,-0.151192
ACH-000005,-0.097058,0.241606,0.193671,0.156428,-0.195506,-0.337316,0.247947,-0.587806,-0.070959,0.004288,...,-0.212524,-0.410596,-0.180655,-0.080401,0.220940,-0.076012,-0.106125,0.068227,0.022568,-0.261572
ACH-000007,0.066212,0.073850,-0.065867,0.156038,0.100958,0.140997,0.066367,-0.472706,-0.008810,0.293417,...,-0.096162,-0.277202,-0.058930,0.108511,0.212320,-0.011173,-0.352653,0.084244,-0.387673,-0.439552
ACH-000009,0.097872,0.000361,-0.055818,0.046221,0.063069,-0.048932,0.070910,-0.640566,0.148335,0.056188,...,-0.293175,-0.259388,-0.095524,0.049911,0.072002,0.011488,-0.596006,0.189440,-0.117177,-0.539860
ACH-000011,0.270232,0.072105,0.014605,0.433243,-0.030273,-0.239080,0.097866,-0.419689,0.138635,0.106322,...,-0.401794,-0.507831,-0.136369,-0.115541,0.274140,0.154517,-0.234775,0.124355,-0.250505,-0.372498


In [29]:
# Filter out this problematic outlier stomach cell line
#crispr_df = crispr_df[crispr_df.index != 'ACH-000167']

In [30]:
recalculate = False
filename = 'crispr_correlations'
filepath = join(basedir, filename)
if recalculate:
    %time crispr_corr = crispr_df.corr()
    crispr_corr.to_hdf('%s.h5' % filepath, filename)
else:
    crispr_corr = pd.read_hdf('%s.h5' % filepath)    

In [31]:
crispr_mean = crispr_corr.values.mean()
crispr_sd = crispr_corr.values.std()
crispr_z = (crispr_corr - crispr_mean) / crispr_sd

In [32]:
crispr_z.head()

,A1BG,A1CF,A2M,A2ML1,A3GALT2,A4GALT,A4GNT,AAAS,AACS,AADAC,...,ZWILCH,ZWINT,ZXDA,ZXDB,ZXDC,ZYG11A,ZYG11B,ZYX,ZZEF1,ZZZ3
A1BG,18.191784,-1.242018,1.360831,-1.011252,-0.002266,1.028501,-0.013928,0.189783,-0.111264,1.054949,...,0.266351,-0.163454,-0.612117,0.116316,-0.993662,0.847710,0.822466,0.866504,0.848916,0.824848
A1CF,-1.242018,18.191784,0.378603,-0.350676,-0.059957,-1.832413,-0.058989,0.879592,-1.663634,0.416674,...,-1.698830,-0.212665,1.627540,1.389030,-0.941134,0.047841,0.625232,-1.342363,-0.415734,0.622525
A2M,1.360831,0.378603,18.191784,1.671774,-0.456240,-1.044572,0.110470,-0.732363,0.035439,1.188912,...,-0.216869,-1.055333,-0.134216,-1.155436,0.038226,-0.533334,0.039291,-0.626993,1.098631,0.728893
A2ML1,-1.011252,-0.350676,1.671774,18.191784,-1.025899,-0.367071,-0.419824,-0.859755,-0.084455,-0.423822,...,-2.486677,-1.683715,-1.694156,-1.419016,0.350669,0.275146,0.245732,-0.222022,-0.919872,-0.640108
A3GALT2,-0.002266,-0.059957,-0.456240,-1.025899,18.191784,-0.243064,-0.702458,-1.097170,-0.273833,0.063425,...,-0.648885,-0.387377,0.292008,0.291964,-2.814227,0.775090,0.761896,-0.005202,-0.379809,1.055833


### Combine CRISPR and RNAi Z-scores

Take the average of CRISPR and RNAi z-scores and drop missing values (that arise from a gene being knocked down/out in one dataset but not the other).

In [33]:
dep_z = (crispr_z + rnai_z) / 2
#dep_z = dep_z.dropna(axis=0, how='all').dropna(axis=1, how='all')

In [34]:
dep_z.head()

,A1BG,A1CF,A2M,A2ML1,A3GALT2,A4GALT,A4GNT,AAAS,AACS,AADAC,...,ZWILCH,ZWINT,ZXDA,ZXDB,ZXDC,ZYG11A,ZYG11B,ZYX,ZZEF1,ZZZ3
A1BG,15.463673,-1.069459,0.501264,-0.749312,NaN,0.427838,0.065514,0.348175,-0.679073,0.676983,...,0.903604,-0.509278,-0.244109,-0.181828,-0.641325,0.599607,NaN,0.302438,0.309948,0.158036
A1CF,-1.069459,15.463673,0.027066,0.431180,NaN,-1.241620,0.206729,0.970342,-0.529760,0.750996,...,-1.508243,-0.364256,0.499767,0.859637,-0.201751,0.800503,NaN,-0.083901,-0.649245,-0.208967
A2M,0.501264,0.027066,15.463673,0.055449,NaN,-0.915802,0.556658,-0.563435,-0.720257,-0.085417,...,-0.678895,-0.290741,-0.682953,-0.218508,-0.934303,-0.288072,NaN,-0.325707,0.520424,-0.389817
A2ML1,-0.749312,0.431180,0.055449,15.463673,NaN,0.047235,-0.229197,0.066786,0.459282,0.434633,...,-1.249283,-0.802965,-0.141557,-1.105889,0.242465,0.050851,NaN,-0.477857,-0.843290,-0.846143
A3GALT2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [35]:
filename = 'dep_z'
filepath = join(basedir, filename)
dep_z.to_hdf('%s.h5' % filepath, filename)

### Expression Data

Load the RNA-seq data (in transcripts per million, obtained from the DepMap website at https://depmap.org/portal/download/). Normalize column names (which include both gene names and Ensemble IDs) to only gene names, and then drop duplicate columns, keeping the first instance of the column for the gene name (there are 87 genes associated with more than one Ensemble ID, and these result in duplicate columns after normalizing to gene names).

In [36]:
%time ccle_df = pd.read_csv(join(basedir, 'CCLE_expression.csv'), index_col=0)

CPU times: user 21.8 s, sys: 677 ms, total: 22.5 s
Wall time: 22.8 s


In [37]:
gene_cols = ['%s' % col.split(' ')[0] for col in ccle_df.columns]
ccle_df.columns = gene_cols
# NOTE: This first way doesn't work! Probably because with duplicate column names a
# separate integer index is created after transposition (?)
#%time ccle_df_drop = ccle_df.T.drop_duplicates(keep='first').T
# See: https://stackoverflow.com/questions/14984119/python-pandas-remove-duplicate-columns
ccle_df_drop = ccle_df.loc[:,~ccle_df.columns.duplicated()]

Calculate expression correlation matrix (takes about 11 min):

In [38]:
recalculate = False
filename = 'expr_correlations'
filepath = join(basedir, filename)
if recalculate:
    %time expr_corr = ccle_df.corr()
    expr_corr.to_hdf('%s.h5' % filepath, filename)
else:
    expr_corr = pd.read_hdf('%s.h5' % filepath)    

Filter the expression data to contain only those columns also contained in the CRISPR dependency data (protein coding genes, for the most part).

In [ ]:
crispr_cols = [c for c in crispr_df.columns]
# Create a boolean mask to filter the CCLE expression data columns
ccle_col_mask = np.array(ccle_df_drop.columns.map(lambda x: x in crispr_cols), dtype=bool)
ccle_df_filt = ccle_df_drop[ccle_df_drop.columns[ccle_col_mask]]

Join the two datasets on the cell line index:

In [ ]:
crispr_expr_join_df = crispr_df.join(ccle_df_filt, how='left', lsuffix=' KO', rsuffix=' RNA')

Calculate the correlations (slow) or reload from HDF5.

In [ ]:
recalculate = False
filename = 'crispr_expr_corr'
filepath = join(basedir, filename)
if recalculate:
    %time ce_corr = crispr_expr_join_df.corr()
    ce_corr.to_hdf('%s.h5' % filepath, filename)
else:
    ce_corr = pd.read_hdf('%s.h5' % filepath)

### Load Drug Data

In [ ]:
#drug_resp_df = pd.read_csv(join(basedir, 'primary-screen-replicate-collapsed-logfold-change.csv'), index_col=0)
#drug_info_df = pd.read_csv(join(basedir, 'primary-screen-replicate-collapsed-treatment-info.csv'), index_col=0)

Join drug sensitivity to expression on the cell line index:

In [ ]:
#drug_expr_df = drug_resp_df.join(ccle_df_filt, how='left')

## Define functions for exploring and visualizing the data

Get the sorted list of correlations for a given gene.

In [ ]:
def sort_corrs(corrs, col_name):
    s = corrs[col_name]
    return sorted(list(zip(s.index, s)), key=lambda x: abs(float(x[1])), reverse=True)

Define a function to get the top correlated genes for a given gene.

In [ ]:
def get_expr_corrs(df, geneX):
    mygene_arr = df[geneX].values
    data = df.values
    corrs = np.zeros(data.shape[1])
    for i in range(data.shape[1]):
        vect = data[:,i]
        r = np.corrcoef(mygene_arr, vect)[0, 1]
        corrs[i] = r
    genes = [c for c in df.columns]
    corr_genes = sorted(list(zip(genes, corrs)), reverse=True, key=lambda x: abs(x[1]))
    corr_genes_dict = {k: v for k, v in corr_genes}
    return corr_genes

In [ ]:
def corrs_for_genes(genes, df=None, cutoff=3.0):
    all_corrs = []
    if df is None:
        df = dep_z
    for gene_a in genes:
        try:
            gene_corrs = sort_corrs(df, gene_a)[1:]
        except KeyError:
            continue
        for gene_b, corr_z in gene_corrs:
            if abs(corr_z > cutoff):
                all_corrs.append((gene_a, gene_b, corr_z))
    return all_corrs

In [ ]:
sort_corrs(expr_corr, 'LATS1')

Define a function to plot the relationship between the transcription of two genes, along with a linear regression and the residuals.

In [ ]:
def scatter(df, geneX, y_arr, y_label, bins=30):
    plt.figure(figsize=(12,3))
    plt.subplot(1, 3, 1)
    plt.plot(df[geneX], y_arr, linestyle='', marker='.')
    plt.xlabel('%s TPM' % geneX)
    plt.ylabel(y_label)
    r = np.corrcoef(df[geneX], y_arr)[0][1]
    plt.title('Corr: %.3f' % r)
    slope, intercept, r_value, p_value, std_err = \
                    stats.linregress(df[geneX], y_arr)
    yhat = slope * df[geneX] + intercept
    plt.plot(df[geneX], yhat, color='k')
    residuals = y_arr - yhat
    plt.subplot(1, 3, 2)
    plt.hist(residuals, bins=bins)
    plt.title('Residuals')
    plt.subplot(1, 3, 3)
    stats.probplot(residuals, dist='norm', plot=plt)
    plt.title('Quantile-quantile plot vs. normal')
    return residuals

In [ ]:
res1 = scatter(ccle_df, 'FOXO3', ccle_df['BAX'], 'AKT1 TPM', bins=20)

In [ ]:
res2 = scatter(ccle_df, 'YAP1', res1, 'Res 1', bins=20)

Next steps:
1. load the INDRA SIF statement file
2. tabulate explanations for correlations;
3. predict correlations from graph structure, edge properties

In [ ]:
sorted(corrs_for_genes(['KIRREL1'], df=expr_corr), 
       key=lambda x: x[2], reverse=True)

## Load Solute Carrier Family genes

In [ ]:
slc_df = pd.read_csv('data/slc_genes.csv')

In [ ]:
slc_genes = list(slc_df['Approved symbol'].values)

In [ ]:
inet_file = '/Users/johnbachman/Dropbox/1johndata/Knowledge File/Biology/Research' \
            '/Big Mechanism/datasets/depmap_analysis/graphs/2021-01-06/indranet_dir_graph.pkl'
with open(inet_file, 'rb') as f:
    inet = pickle.load(f)

In [5]:
# Load reactome dict
reactome_file = '/Users/johnbachman/Dropbox/1johndata/Knowledge File/Biology/Research/Big Mechanism/' \
                'datasets/depmap_analysis/reactome_pathways.pkl'

apriori_mapping = get_mitocarta_info(mito_file)

with open(reactome_file, 'rb') as f:
    up2path, _, pathid2pathname = pickle.load(f)
    
reactome_dict = {'uniprot_mapping': up2path,
                 'pathid_name_mapping': pathid2pathname}

expl_funcs=['apriori_explained', 'expl_ab', 'expl_ba', 'expl_axb',
            'expl_bxa', 'find_cp', 'get_st', 'common_reactome_paths']

sd_range = (2, None)
dme_file = 'slc_explainer.pkl'

In [ ]:
run_depmap(inet, dme_file, 'unsigned', sd_range, subset_list=slc_genes,
           z_score=dep_z, overwrite=True, reactome_dict=reactome_dict,
           depmap_date='21q1', expl_funcs=expl_funcs, apriori_explained=apriori_mapping, n_chunks=4)

In [ ]:
import mplayer
p = mplayer.Player()
success_file = '/Users/johnbachman/Downloads/success.m4a'
p.loadfile(success_file)

In [6]:
with open(dme_file, 'rb') as f:
    dme = pickle.load(f)

In [7]:
dme_pair = dme.stats_df.set_index('pair').drop(['agA', 'agB', 'z_score'], axis=1)
dme_df = dme.expl_df.join(dme_pair, on='pair')

In [8]:
slc_to_expl = dme_df[(dme_df['z_score'] > 3) &
                     (dme_df['apriori_explained'] == False) &
                     (dme_df['reactome_paths'] == False)]

In [17]:
dme.plot_corr_stats('slc_output', z_corr=dep_z)

NameError: name 'dep_z' is not defined

In [13]:
gene = 'SLC4A7'
gene_df = slc_to_expl[(slc_to_expl['agA'] == gene) | (slc_to_expl['agB'] == gene)]
expl_set = []
for row in gene_df.itertuples():
    if row.expl_type == 'shared_regulator':
        continue
    elif row.expl_type == 'shared_target':
        expl_data = row.expl_data[2]
    else:
        expl_data = row.expl_data
    for expl in expl_data:
        expl_set.append(expl)
expl_ctr = Counter(expl_set)
expl_ctr = sorted([(k, v) for k, v in expl_ctr.items()], key=lambda x: x[1], reverse=True)

In [14]:
expl_ctr

[('NTRK1', 43),
 ('TRIM25', 30),
 ('cell population proliferation', 24),
 ('apoptotic process', 22),
 ('sodium atom', 19),
 ('TMEM17', 18),
 ('Hydrogen-Ion Concentration', 15),
 ('Neoplasm Invasiveness', 14),
 ('cell differentiation', 14),
 ('magnesium atom', 13),
 ('LGALS3', 12),
 ('calcium(2+)', 12),
 ('cell cycle', 10),
 ('transport', 10),
 ('reactive oxygen species', 9),
 ('CDH1', 9),
 ('ERK', 9),
 ('TMEM216', 9),
 ('PI3K', 8),
 ('TGFB1', 8),
 ('ELAVL1', 8),
 ('ESR1', 8),
 ('localization', 8),
 ('SRC', 8),
 ('sodium chloride', 7),
 ("3',5'-cyclic AMP", 7),
 ('EGFR', 7),
 ('PRKAA1', 7),
 ('ERBB2', 7),
 ('CXCL8', 6),
 ('AR', 6),
 ('homeostatic process', 6),
 ('CSF2', 6),
 ('GPI', 6),
 ('RACK1', 6),
 ('hydrogen chloride', 6),
 ('BCAT1', 6),
 ('KLF4', 5),
 ('AKT1', 5),
 ('SLC9A1', 5),
 ('CA2', 4),
 ('SNX21', 3),
 ('PKA', 3),
 ('progesterone', 3),
 ('pyraclofos', 3),
 ('PPP3', 3),
 ('UNK', 3),
 ('CSF1', 3),
 ('GNAS', 3),
 ('RAS', 3),
 ('YWHAB', 3),
 ('bisphenol A', 3),
 ('cooH', 3),
 ('

## Other

In [ ]:
from indra.util import batch_iter

In [ ]:
goi = 'MDM2'
res = sort_corrs(dep_z, goi)
#for r in res[0:20]:
#    print(r)
expl_pct = []
batch_size = 1000
for batch in batch_iter(res, batch_size):
    counter = 0
    for gene, corr in batch:
        if gene in indra_dg[goi]:
            counter += 1
    pct = 100 * (counter / batch_size)
    expl_pct.append(pct)

In [ ]:
plt.figure()
plt.bar(range(len(expl_pct)), expl_pct)

# Drug-Drug Correlations

In [ ]:
drug2_corrs = drug_resp_df.corr()

In [ ]:
from indra.sources import tas
tp = tas.process_from_web()

In [ ]:
tas_stmts = tp.statements

In [ ]:
# Build up a dictionary of mappings from CHEBI to SMILES and INCHIKEY
import pyobo
chebi_inchikey_property = 'http://purl.obolibrary.org/obo/chebi/inchikey'
chebi_smiles_property = 'http://purl.obolibrary.org/obo/chebi/smiles'
chebi_id_to_smiles = pyobo.get_filtered_properties_mapping('chebi', chebi_smiles_property)
chebi_id_to_inchikey = pyobo.get_filtered_properties_mapping('chebi', chebi_inchikey_property)

In [ ]:
foo = [stmt for stmt in tas_stmts if stmt.agent_list()[0].name == 'vemurafenib']

In [ ]:
foo[0]

In [ ]:
foo[0].subj.db_refs

In [ ]:
chembl_ids = [stmt.agent_list()[0].db_refs.get('CHEMBL') for stmt in tas_stmts]
chembl_ids = list(set([i for i in chembl_ids if i]))

In [ ]:
len(chembl_ids)

In [ ]:
chembl_ids_str = ';'.join(chembl_ids[0:200])

In [ ]:
j = res.json()

In [ ]:
j['molecules'][0]['molecule_chembl_id']

In [ ]:
import requests
from indra.util import batch_iter
batch_size = 200
chembl_url = 'https://www.ebi.ac.uk/chembl/api/data/molecule/set/'
inchi_keys = []
for ix, batch in enumerate(batch_iter(chembl_ids, batch_size)):
    chembl_ids_str = ';'.join(list(batch))
    print(ix)
    res = requests.get(chembl_url + chembl_ids_str, headers={'Accept': 'application/json'})
    if res.status_code != 200:
        print("Error occurred")
        break
    for mol in j['molecules']:
        cid = mol['molecule_chembl_id']
        ik = mol['molecule_structures']['standard_inchi_key']
        inchi_keys.append((cid, ik))

In [ ]:
j = res.json()

In [ ]:
inchi_to_chembl = {t[1]: t[0] for t in inchi_keys}
chembl_to_inchi = {t[0]: t[1] for t in inchi_keys}

In [ ]:
def get_drug_id(name):
    drug_rec = drug_info_df[drug_info_df.name == name]
    return drug_rec.index.to_list()[0]

def get_drug_name(drug_id):
    drug_rec = drug_info_df.loc[drug_id]
    return drug_rec['name']

def get_drug_targets(drug_id):
    drug_rec = drug_info_df.loc[drug_id]
    targets = drug_rec['target']
    if pd.isna(targets):
        return []
    else:
        return [t.strip() for t in targets.split(',')]

In [ ]:
# Dump all smiles strings from the DepMap data
depmap_smiles = set()
for _, smiles_str in drug_info_df.smiles.items():
    if pd.isna(smiles_str):
        continue
    for sm in smiles_str.split(','):
        sm = sm.strip()
        if sm:
            depmap_smiles.add(sm.strip())
            
depmap_smiles = list(demap_smiles)

with open('depmap_smiles.txt', 'wt') as f:
    for sm in depmap_smiles:
        f.write('%s\n' % sm)

In [ ]:
# Convert from SMILES to INCHIKEY using obabel:
# $ obabel -ismi depmap_smiles.txt -oinchikey > depmap_inchikey.txt

In [ ]:
# Now, read the INCHIKEY databack in and build up a mapping dictionary
depmap_inchikey = []
with open('depmap_inchikey.txt', 'rt') as f:
    depmap_inchikey = [l.strip() for l in f.readlines()]
depmap_smiles_to_ik = dict(zip(depmap_smiles, depmap_inchikey))

In [ ]:
import requests
from indra.util import batch_iter
chembl_url = 'https://www.ebi.ac.uk/chembl/api/data/substructure/'
dmik_to_chembl = {}
for ix, ik in enumerate(depmap_smiles_to_ik.values()):
    if ix > 10:
        break
    res = requests.get(chembl_url + ik, headers={'Accept': 'application/json'})
    if res.status_code != 200:
        print("Error occurred")
        break
    if res.json()['molecules']:
        for mol in res.json()['molecules']:
            cid = mol['molecule_chembl_id']
            if ik not in dmik_to_chembl:
                dmik_to_chembl[ik] = [cid]
            else:
                dmik_to_chembl[ik].append(cid)

In [ ]:
dmik_to_chembl

In [ ]:
drug_name = 'vemurafenib'

In [ ]:
drug_info_df[drug_info_df['name'] == 'vemurafenib']

In [ ]:
sm = [s.strip() for s in drug_info_df[drug_info_df['name'] == 'vemurafenib']['smiles'].values[0].split(',')]

In [ ]:
sm

In [ ]:
for s in sm:
    ik = depmap_smiles_to_ik[s]
    print(ik)

In [ ]:
chembl_to_inchi['CHEMBL1229517']

In [ ]:
drug_name = 'abemaciclib'
corrs = sort_corrs(drug2_corrs, get_drug_id(drug_name))
results = []
corr_k = 20
for i, (corr_drug_id, corr_val) in enumerate(corrs):
    if corr_val == 1.0:
        continue
    if i >= corr_k:
        break
    corr_drug_name = get_drug_name(corr_drug_id)
    corr_targets = get_drug_targets(corr_drug_id)
    for t in corr_targets:
        results.append((corr_drug_name, corr_drug_id, corr_val, t))
tgt_df = pd.DataFrame.from_records(results, columns=('drug_name', 'drug_id', 'corr', 'target'))

In [ ]:
tgt_df

In [ ]:
tgt_df

In [ ]:
tgt_corrs = tgt_df.groupby('target')['corr'].sum()
tgt_counts = tgt_df.groupby('target')['drug_id'].count()
tgt_corrs.sort_values(ascending=False, inplace=True)
tgt_counts.sort_values(ascending=False, inplace=True)

In [ ]:
num_tgts = 20
top_corrs = tgt_corrs[0:num_tgts]
labels = top_corrs.index.to_list()
corrs = top_corrs.values

In [ ]:
plt.figure()
plt.bar(range(len(corrs)), corrs, align='center', tick_label=labels)
plt.xticks(rotation='vertical')
plt.subplots_adjust(bottom=0.2)
plt.xlabel('Nominal targets of top %d correlates' % corr_k)
plt.ylabel('Count')
plt.title('Targets of drugs correlated with %s' % drug_name)
plt.show()

# Depmap Paper Figures

## Figure 1: Example correlation

In [ ]:
from indra.util import plot_formatting as pf
from scipy.stats import linregress

pf.set_fig_params()
geneA = 'CDKN1A'
geneB = 'CHEK2'
lw = 0.5
ms = 1
fig = plt.figure(figsize=(4, 2))
# CRISPR plot
plt.subplot(1, 2, 1)
ax = plt.gca()
ax.axhline(y=0, color='k', linewidth=lw)
ax.axvline(x=0, color='k', linewidth=lw)
plt.plot(crispr_df[geneA].values, crispr_df[geneB].values, marker='.', markersize=ms, linestyle='')
# Plot linear regression
crispr_lr = linregress(crispr_df[geneA].values, crispr_df[geneB].values)
plt.plot(crispr_df[geneA].values, crispr_df[geneA].values * crispr_lr.slope + crispr_lr.intercept,
         linestyle='-', linewidth=lw, color='black')
plt.xlabel(f'{geneA} Gene Effect (CERES)')
plt.ylabel(f'{geneB} Gene Effect (CERES)')
ax.text(1.4, 1.5, 'rho = %0.3f' % crispr_lr.rvalue, fontsize=pf.fontsize)
ax.text(1.4, 1.35, 'z = %0.2f' % crispr_z[geneA][geneB], fontsize=pf.fontsize)
pf.format_axis(ax)
# RNAi plot
plt.subplot(1, 2, 2)
rnai_df_filt = rnai_df[~pd.isnull(rnai_df[geneA]) & ~pd.isnull(rnai_df[geneB])]
ax = plt.gca()
ax.axhline(y=0, color='k', linewidth=lw)
ax.axvline(x=0, color='k', linewidth=lw)
plt.plot(rnai_df_filt[geneA].values, rnai_df_filt[geneB].values, marker='.', markersize=ms, linestyle='')
# Plot linear regression
rnai_lr = linregress(rnai_df_filt[geneA].values, rnai_df_filt[geneB].values)
plt.plot(rnai_df_filt[geneA].values, rnai_df_filt[geneA].values * rnai_lr.slope + rnai_lr.intercept,
         linestyle='-', linewidth=lw, color='black')
plt.xlabel(f'{geneA} Gene Effect (DEMETER2)')
plt.ylabel(f'{geneB} Gene Effect (DEMETER2)')
ax.text(0.7, 0.69, 'rho = %0.3f' % rnai_lr.rvalue, fontsize=pf.fontsize)
ax.text(0.7, 0.57, 'z = %0.2f' % rnai_z[geneA][geneB], fontsize=pf.fontsize)

pf.format_axis(ax)
fig.tight_layout()
plt.savefig(join(figdir, f'{geneA}_{geneB}_corr_plot.pdf'))
dep_z[geneA][geneB]

## Figure 2: Stronger Effects -> Stronger Correlations, Mito genes

In [ ]:
mitocarta = pd.read_excel('data/Human.MitoCarta2.0.xls', sheet_name=1)
mitogenes = list(mitocarta.Symbol.values)

In [ ]:
len(mitogenes)

In [ ]:
'SSBP1' in mitogenes

In [ ]:
crispr_mean = crispr_df.abs().mean(axis=0).sort_values()

In [ ]:
crispr_z_mean = crispr_z.abs().mean(axis=0)

In [ ]:
n_corrs = 100
np.mean(sorted(crispr_z['CHEK2'].abs(), reverse=True)[1:n_corrs+1])

In [ ]:
#crispr_z_mean = crispr_z.apply(lambda x: np.mean(sorted(x.abs(), reverse=True)[1:101]), axis=1)

In [ ]:
crispr_z_mean.CHEK2

In [ ]:
effect_corr = pd.concat([crispr_mean, crispr_z_mean], axis=1, sort=False)

In [ ]:
mito_effect = effect_corr[effect_corr.index.isin(mitogenes)]
nonmito_effect = effect_corr[~effect_corr.index.isin(mitogenes)]

In [ ]:
res = nonmito_effect.groupby(pd.qcut(nonmito_effect[0].values, 400)).mean()

In [ ]:
plt.plot(nonmito_effect[0], nonmito_effect[1], linestyle='', marker='.', color='b', alpha=0.5, label='Not in MitoCarta')
plt.plot(mito_effect[0], mito_effect[1], linestyle='', marker='.', color='r', alpha=0.5, label='In MitoCarta')
plt.plot(res[0].values, res[1].values, linewidth=3, color='black')
plt.xlabel('Average CRISPR Gene Effect')
plt.ylabel(f'Avg of top {n_corrs} CRISPR Corrs (z-scores)')
plt.legend()

## Clustering Correlation Data

In [ ]:
dep_z.columns

# Deprecated/Old

## Load Cosmic Data

In [ ]:
cos_df = pd.read_csv('data/Census_allThu Jun 13 20_37_30 2019.csv')

In [ ]:
cos_df.head()

In [ ]:
cos_corrs = corrs_for_genes(cos_df['Gene Symbol'])

In [ ]:
dme

In [ ]:
5+5

In [ ]:
slc_df.head()
slc_corrs = corrs_for_genes(slc_df['Approved symbol'])

In [ ]:
slc_df

In [ ]:
mitocarta = pd.read_excel('data/Human.MitoCarta2.0.xls', sheet_name=1)
mitogenes = list(mitocarta.Symbol.values)

In [ ]:
'PNPT1' in mitogenes

In [ ]:
slc_corrs

In [ ]:
slc_corrs_filt = []
slc_corrs_seen = set()
for corr in slc_corrs:
    # Filter out correlations we've already seen
    corr_set = frozenset(corr)
    if corr_set in slc_corrs_seen:
        continue
    else:
        slc_corrs_seen.add(corr_set)
    # Filter out self-correlations
    if corr[0] == corr[1]:
        continue
    # Filter out mitochondrial gene correlations
    elif corr[0] in mitogenes or corr[1] in mitogenes:
        continue
    else:
        slc_corrs_filt.append(corr)

In [ ]:
slc_corrs_filt.sort(key=lambda x: x[2], reverse=True)

In [ ]:
slc_corrs_filt

In [ ]:
len(slc_corrs_filt)

In [ ]:
import csv
with open('slc_corrs_no_mito.csv', 'wt') as f:
    csv_writer = csv.writer(f, delimiter=',')
    csv_writer.writerows(slc_corrs_filt)

In [ ]:
import requests

In [ ]:
query = {'source': None,
         'target': None,
         'stmt_filter': ['conversion', 'fplx'],
         'node_filter': ['hgnc', 'fplx', 'chebi', 'pubchem', 'go', 'mesh'],
         'node_blacklist': [],
         'edge_hash_blacklist': [],
         'path_length': 1,
         'sign': 'no_sign',
         'weighted': False,
         'bsco': 0.,
         'direct_only': False,
         'curated_db_only': False,         
         'fplx_expand': False,
         'simple': False,
         'k_shortest': False}

def get_expl_corrs(gene_pairs):
    no_path = []
    has_2path = []
    has_3path = []
    for ix, (gene_a, gene_b, z_score) in enumerate(gene_pairs):
        for source, target in ((gene_a, gene_b), (gene_b, gene_a)):
            print("%d getting paths for %s, %s" % (ix, source, target))
            query['source'] = source
            query['target'] = target
            query['path_length'] = 1
            res = requests.post('http://127.0.0.1:5000/query/submit', json=query).json()
            if '2' in res['result']['paths_by_node_count']:
                has_2path.append((source, target, z_score))
            else:
                query['path_length'] = 2
                res = requests.post('http://127.0.0.1:5000/query/submit', json=query).json()
                if '3' in res['result']['paths_by_node_count']:
                    has_3path.append((source, target, z_score))                
                else:
                    no_path.append((source, target, z_score))
    return {'no_path': no_path, 'has_2path': has_2path, 'has_3path': has_3path}

In [ ]:
paths = get_expl_corrs(sorted(slc_corrs, key=lambda x: x[2], reverse=True))

In [ ]:
paths['has_2path']

In [ ]:
paths['has_3path']

In [ ]:
paths['no_path']

In [ ]:
query['source'] = 'GRSF1'
query['target'] = 'SLC30A9'
query['path_length'] = 2
res = requests.post('http://127.0.0.1:5000/query/submit', json=query).json()

In [ ]:
res

## Load INDRA SIF DB

In [ ]:
stmts_df = pd.read_csv('data/stmts_by_pair_type.csv')
# Filter to HGNC only
stmts_df = stmts_df[(stmts_df['agA_ns'] == 'HGNC') & (stmts_df['agB_ns'] == 'HGNC')]
# Remove namespace and identifier columns, leaving only name
stmts_df = stmts_df[['agA_name', 'agB_name', 'stmt_type', 'evidence_count']]
# Remove self-edges
stmts_df = stmts_df[stmts_df['agA_name'] != stmts_df['agB_name']]
# Useful bits of Pandas to know, ultimately not used here:
# pd.crosstab([foo['agA_name'], foo['agB_name']], foo['stmt_type'], foo['evidence_count'],
#             aggfunc='sum', dropna=False).fillna(0)
# baz = bar[bar.apply(lambda x: x.name[0] != x.name[1], axis=1)]

In [ ]:
x_dict = {}
for row in stmts_df.values:
    try:
        raw_corr = dep_z[row[0]][row[1]]
        """
        if int(raw_corr) == 0:
            corr = 0
        elif int(raw_corr) < 0:
            corr = -1
        elif int(raw_corr) > 0:
            corr = 1
        """
        corr = raw_corr
    except KeyError:
        continue
    stmt_type = row[2]
    gene_pair = (row[0], row[1])
    if gene_pair not in x_dict:
        x_dict[gene_pair] = {'Corr': corr}
    x_dict[gene_pair][stmt_type] = row[3]
features = ['Activation', 'Inhibition', 'Corr']
all_data = np.zeros((len(x_dict), len(features)))
for row_ix, feat_dict in enumerate(x_dict.values()):
    for feat_name, feat_val in feat_dict.items():
        if feat_name not in features:
            continue
        feat_ix = features.index(feat_name)
        all_data[row_ix, feat_ix] = feat_val

In [ ]:
"""
# Shuffle the matrix in place
np.random.shuffle(all_data)
# Balance the data
ctr = Counter(all_data[:, -1])
classes = sorted([(k, v) for k,  v in ctr.items()], key=lambda x: x[0])
# Get the class with the fewest members
min_class_count = min([t[1] for t in classes])
min_class_count
data_bal = np.zeros((min_class_count * len(classes), len(features)))
row_ix = 0
for one_class, count in classes:
    num_class_rows = 0
    for ix in range(all_data.shape[0]):
        if all_data[ix, -1] == one_class:
            data_bal[row_ix] = all_data[ix,:]
            row_ix += 1
            num_class_rows += 1
        if num_class_rows >= min_class_count:
            print("Finished class %s, at rows %s" % (one_class, row_ix))
            break

"""

In [ ]:
# Divide training and test
np.random.shuffle(all_data)
partition_ix = int(len(all_data) * 0.8)
train_data = all_data[0:partition_ix, :]
test_data = all_data[partition_ix:, :]
x_train = train_data[:,:-1]
y_train = train_data[:, -1]
x_test = test_data[:, :-1]
y_test = test_data[:, -1]

## Multinomial Naive Bayes

In [ ]:
from sklearn.naive_bayes import MultinomialNB
nb = MultinomialNB()
nb.fit(x_train, y_train)
nb.score(x_test, y_test)

In [ ]:
yctr = Counter(y_train)
yclasses = sorted([(k, v) for k,  v in yctr.items()], key=lambda x: x[0])

In [ ]:
[features + ['Count']] + list(zip(nb.coef_, classes))

## Linear Regression

In [ ]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import explained_variance_score
from sklearn.kernel_ridge import KernelRidge
lr = LinearRegression()

In [ ]:
lr.fit(x_train, y_train)
lr.coef_

In [ ]:
y_pred = lr.predict(x_train)
explained_variance_score(y_train, y_pred)

In [ ]:
y_mean = np.full(y_train.shape, y_train.mean())
explained_variance_score(y_train, y_mean)

In [ ]:
y_pred_test = lr.predict(x_test)
explained_variance_score(y_test, y_pred_test)

## Load INDRA Statements (deprecated)

In [ ]:
reload = False
if reload:
    inc_stmts = ac.load_statements('increase_amt.pkl')
    dec_stmts = ac.load_statements('decrease_amt.pkl')
    stmts = inc_stmts + dec_stmts
    stmts = ac.map_grounding(stmts)
    stmts = [s for s in stmts if s.subj and s.subj.db_refs.get('HGNC')
                             and s.obj and s.obj.db_refs.get('HGNC')]
    stmts = ac.run_preassembly(stmts)
    ac.dump_statements(stmts, 'assembled_stmts.pkl')
else:
    stmts = ac.load_statements('assembled_stmts.pkl')

In [ ]:
stmts_by_obj = defaultdict(list)
for s in stmts:
    gene_id = s.obj.db_refs['HGNC']
    gene_name = hgnc_client.get_hgnc_name(gene_id)
    stmts_by_obj[gene_name].append(s)

In [ ]:
stmt_counts = []
for obj, stmts in stmts_by_obj.items():
    stmt_counts.append((obj, len(stmts)))
stmt_counts.sort(key=lambda x: x[1], reverse=True)

In [ ]:
stmt_counts[0:10]

In [ ]:
gene = 'BCL2'
rank = 0
stmts_by_obj[gene].sort(key=lambda s: s.belief, reverse=True)
print(stmts_by_obj[gene][rank], '\n')
print('\n'.join(['%d: %s\n' % (i, str((e.source_api, e.text)))
                 for i, e in enumerate(stmts_by_obj[gene][rank].evidence)]))